### Extract device settings as CSV files for `Settings Manager` from an XLSX file prepared for the beam envelope controling during the cavity phase scan tuning.

In [1]:
import xlrd
from phantasy import MachinePortal
from phantasy import disable_warnings
from phantasy_apps.settings_manager.contrib import SettingsRow

disable_warnings()

In [2]:
# input parameters for LS1 cavity tuning
machine = "FRIB"
segments = ["MEBT_FS1A"]

xlsx_filepath = "20200224_phase_scan_setting_list_maruta.xlsx"
sheet_name = "36Ar10 magnets"
row_start = 4
column_start = 4
settings_rootpath = "extracted_settings_LS1"

In [2]:
# input parameters for LS3 cavity tuning
machine = "FRIB"
segments = ["MEBT_FS1B", "STRIPPER_FS2A"]

xlsx_filepath = "ARR04_phase_scan_ver20200316.xlsx"
sheet_name = "36Ar LS2 phase scan"
row_start = 4
column_start = 4
settings_rootpath = "extracted_settings_LS2"

In [7]:
wkbk = xlrd.open_workbook(xlsx_filepath)
if sheet_name not in wkbk.sheet_names():
    raise RuntimeError("Cannot find sheet {} from given xlsx file.".format(sheet_name))
magnets_settings = wkbk.sheet_by_name(sheet_name)
head_row = magnets_settings.row(1)
head_row;

In [8]:
mp = MachinePortal(machine, segments[0]);
for segm in segments[1:]:
    mp.load_lattice(segm)
lat = mp.combined_lattice()
lat;

In [9]:
# get a list of column index of element name and element object.
# [(cell index, CaElement)]
index_elem_list = []
col_idx0 = column_start - 1
for i, cell in enumerate(head_row[col_idx0:], col_idx0):
    if cell.value == 'STOP':
        break
    elem = lat.get_elements(name="*{}*".format(cell.value))
    if elem:
        index_elem_list.append((i, elem[0]))

In [10]:
list_settings_row = []
last_settings_row = None
for ridx in range(row_start - 1, magnets_settings.nrows):
    row = magnets_settings.row(ridx)
    current_settings_row = SettingsRow(row, index_elem_list, last_settings_row)
    if not current_settings_row == last_settings_row:
        current_settings_row.write(rootpath=settings_rootpath, row_id=ridx + 1)
    last_settings_row = current_settings_row

Write settings file 04-LS2 Transport-0-19.796.csv
Write settings file 11-LS2_CC02-1-23.267.csv
Write settings file 29-LS2_CC05-1-38.424.csv
Write settings file 77-LS2_CD01-1-80.584.csv
Write settings file 173-Full acceleration-0-238.265.csv
